<h1> Processing of the yeast data <span class="tocSkip"></span></h1>

# Introduction

The data needs to be processed to reduce the memory required for training, particularly in cases such as statistical or tree-based methods that need to reduce dimensionality to achieve an efficient balance between time, memory, and performance during training. Additionally, various datasets will be generated with different percentages of missing data to evaluate imputation performance across these different cases.

In [25]:
import numpy as np
import pandas as pd  
from matplotlib import pyplot as plt
import zipfile
import builtins  # Para asignar variables globales dinámicamente

### Descomprimir zip y guardar como df

In [26]:


def unzip(archivo_zip, variable_name):
    """
    Extrae y carga el contenido del único archivo dentro de un ZIP en una variable global.
    
    Parámetros:
    - archivo_zip: Ruta al archivo ZIP.
    - variable_name: Nombre de la variable global donde se guardará el DataFrame.
    """
    with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
        # Obtiene directamente el primer (y único) archivo
        file_name = zip_ref.namelist()[0]
        with zip_ref.open(file_name) as txt_file:
            # Leer el archivo como un DataFrame de pandas
            df = pd.read_csv(txt_file, sep='\t', index_col=0)
            
            # Asignar el DataFrame a una variable global con el nombre proporcionado
            builtins.__dict__[variable_name] = df

# Ejemplo de uso
zip_file_train= '../data/raw/yeast_genotype_train.txt.zip'
zip_file_test = '../data/raw/yeast_genotype_test.txt.zip'

# Llamar a la función para descomprimir y guardar los DataFrames en variables globales
unzip(zip_file_train, 'df_train')  # Se guarda en la variable global df_train
unzip(zip_file_test, 'df_test')   # Se guarda en la variable global df_test



In [27]:
# Ahora puedes acceder a df_train y df_test como variables globales
print("Entrenamiento:")
print(df_train.head(5))


print("\nPrueba:")
print(df_test.head(5))
df_train.shape, df_test.shape

Entrenamiento:
      33070_chrI_33070_A_T 33147_chrI_33147_G_T 33152_chrI_33152_T_C  \
SAMID                                                                  
01_01                    1                    1                    1   
01_02                    1                    1                    1   
01_03                    2                    2                    2   
01_04                    1                    1                    1   
01_06                    2                    2                    2   

      33200_chrI_33200_C_T 33293_chrI_33293_A_T 33328_chrI_33328_C_A  \
SAMID                                                                  
01_01                    1                    1                    1   
01_02                    1                    1                    1   
01_03                    2                    2                    2   
01_04                    1                    1                    1   
01_06                    2                    2 

((3513, 28220), (877, 28220))

In [28]:
#examples of data
df_train.head(10)

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_02,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_03,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
01_07,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_08,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_09,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_10,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2


## Reducing dimensionality with MCA
Análisis de Correspondencia Múltiple (MCA) es una técnica que te permitirá reducir la dimensionalidad de tus datos categórico

In [29]:
%pip install prince
import pandas as pd
import prince

# Supongamos que tu df_train es un DataFrame de pandas con las variantes genéticas
# Si no tienes los datos categóricos en tipo `category`, conviértelos
df_train = df_train.astype('category')

# Aplicar Análisis de Correspondencia Múltiple (MCA)
mca = prince.MCA(
    n_components=1000,  # Número de componentes principales
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)


Note: you may need to restart the kernel to use updated packages.


In [33]:
# Ajustar el MCA a los datos de entrenamiento
mca = mca.fit(df_train)

# Transformar los datos originales en el espacio de baja dimensionalidad
df_train_mca = mca.transform(df_train)


TypeError: 'RangeIndex' object is not callable

In [ ]:
# Ver las nuevas dimensiones
print(df_train_mca.columns())

In [31]:
from sklearn.model_selection import train_test_split
df_train_split, df_val_split = train_test_split(df_train, test_size=0.2, random_state=42)

# Method

In [32]:
# Convertir df_test a categoría si es necesario
df_test = df_test.astype('category')

# Aplicar la misma transformación MCA al conjunto de test
df_test_mca = mca.transform(df_test)


def introduce_missingness(data, missing_perc=0.1):
    """
    Introduce valores faltantes (NaN) en un porcentaje determinado de celdas en el DataFrame.
    """
    data_missing = data.copy()

    # Calcular el número de valores a modificar
    n_samples, n_features = data_missing.shape
    n_missing = int(np.floor(n_samples * n_features * missing_perc))

    # Seleccionar aleatoriamente las posiciones donde se introducirán los valores faltantes (NaN)
    missing_indices = np.random.choice(n_samples * n_features, n_missing, replace=False)

    # Introducir los valores faltantes (NaN) en las posiciones seleccionadas
    for index in missing_indices:
        i = index // n_features
        j = index % n_features
        data_missing.iloc[i, j] = np.nan  # Introducir NaN como valor faltante

    return data_missing

# Aplicar la función al conjunto de test
df_test_mca_missing_10 = introduce_missingness(df_test_mca, missing_perc=0.1)
df_test_mca_missing_20 = introduce_missingness(df_test_mca, missing_perc=0.2)
df_test_mca_missing_30= introduce_missingness(df_test_mca, missing_perc=0.3)
df_test_mca_missing_40 = introduce_missingness(df_test_mca, missing_perc=0.4)

In [23]:
df_train_split.columns= df_train_split.columns.map(str)
df_val_split.columns= df_val_split.columns.map(str)
df_test_mca_missing_10.columns= df_test_mca_missing_10.columns.map(str)
df_test_mca_missing_20.columns= df_test_mca_missing_20.columns.map(str)
df_test_mca_missing_30.columns= df_test_mca_missing_30.columns.map(str)
df_test_mca_missing_40.columns= df_test_mca_missing_40.columns.map(str)

### GUARDAR ARCHIVOS RESULTANTES PARA ENTRENAR Y PROBAR MODELOS CON LOS MISMOS DATOS

In [24]:

df_train_split.to_parquet('../data/processed/df_train_reduced.parquet')
df_val_split.to_parquet('../data/processed/df_val_reduced.parquet')
df_test_mca_missing_10.to_parquet('../data/processed/df_test_reduced_missing_10.parquet')
df_test_mca_missing_20.to_parquet('../data/processed/df_test_reduced_missing_20.parquet')
df_test_mca_missing_30.to_parquet('../data/processed/df_test_reducedmissing_30.parquet')
df_test_mca_missing_40.to_parquet('../data/processed/df_test_mca_reduced_40.parquet')


Note: you may need to restart the kernel to use updated packages.
